<h1>Brain Tumor Classification</h1>
<p>SVM model is implemented to classify and detect wether the MRI has a tumor or not and classify which type of tumor
<br>
The features were HOG + LBP + GLCM
</p>

In [ ]:
# utility libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import joblib
from tqdm import tqdm

# image preprocessing libray
import cv2 as cv
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops, hog
from skimage import morphology


# machine learnin model
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
HEIGHT = 128 #bisa coba 64
WIDTH = 128 #bisa coba 64
IMG_SIZE = (HEIGHT, WIDTH)

<h1>Reading Dataset</h1>

In [ ]:
classification_path = "dataset/brisc2025/classification_task/train"
segmentation_path = "dataset/brisc2025/segmentation_task/train"


class loadDataset:
    def __init__(self):
       self.class_int = {}
       self.int_to_class = {}

    def readDataset(self, datapath, img_size):

        self.class_int = {}
        self.int_to_class = {}

        data = []
        labels = []

        for i, img_class in enumerate(sorted(os.listdir(datapath))):

            class_path = os.path.join(datapath, img_class)
            if not os.path.isdir(class_path):
                continue

            self.class_int[img_class] = i
            self.int_to_class[i] = img_class

            for img_path in tqdm(os.listdir(class_path)):

                true_path = os.path.join(datapath, img_class, img_path)

                # read image
                img = cv.imread(true_path)

                # resize imaga
                img = cv.resize(img, dsize=img_size, interpolation=cv.INTER_AREA)
                
                # save the image
                data.append(img)

                # 
                labels.append(i)

        return labels, data
    
    def mapLabel(self):
        
        return self.int_to_class



In [ ]:
loader = loadDataset()
y, x = loader.readDataset(datapath=classification_path, img_size=IMG_SIZE)

In [ ]:
print(f"Length of the dataset: {len(x)}\ndataset shape: {x[0].shape}")

<h1>Preprocessing Image</h1>

In [ ]:
class Preprocessor():
    def __init__(self, ksize=(3, 3), clip_limit=2.0, tile_size=(4, 4)):
        self.ksize = ksize
        self.clip_limit = clip_limit
        self.tile_size = tile_size
        self.clahe = cv.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_size)

    def process_img(self, data):
        
        transformed_data = []

        for img in tqdm(data):

            # grayscale
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

            # smoothing first
            smoothed = cv.GaussianBlur(gray, ksize=self.ksize, sigmaX=0)

            # enhanced using CLAHE
            enhanced = self.clahe.apply(smoothed)

            transformed_data.append(enhanced)

        return transformed_data



In [ ]:
def visualize(img, title=None):

    plt.figure(figsize=(9, 6))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.show()

In [ ]:
preprocess = Preprocessor()
x_preprocessed = preprocess.process_img(x)

In [ ]:
def visualize_comparison(original, preprocessed, idx=0):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(cv.cvtColor(original[idx], cv.COLOR_BGR2RGB))
    plt.title(f"Original Image (Index {idx})")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(preprocessed[idx], cmap='gray')
    plt.title("Preprocessed (Grayscale + CLAHE)")
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
print(f"Dataset length: {len(x_preprocessed)}\nDataset shape: {x_preprocessed[0].shape}")

In [ ]:
visualize(x_preprocessed[0], title='Hasil gambar preprocess')
visualize_comparison(x, x_preprocessed, idx=0)

<h1>Extract Features</h1>

In [ ]:
class ExtractFeature:
    def __init__(self):
        pass
    
    def statistical_feature(self, img):
        mean = np.mean(img)
        std = np.std(img)
        median = np.median(img)
        f_stats = np.array([mean, std, median])

        return f_stats
    
    def glcm_feature(self, img):
        img_glcm = img.astype(np.uint8)

        glcm = graycomatrix(img_glcm,
                            distances=[1],
                            angles=[0],
                            levels=256,
                            symmetric=True,
                            normed=True
                            )
        
        f_contrast = graycoprops(glcm, 'contrast')[0, 0]
        f_energy = graycoprops(glcm, 'energy')[0, 0]
        f_homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
        f_correlation = graycoprops(glcm, 'correlation')[0, 0]
        f_glcm = np.array([f_contrast, f_energy, f_homogeneity, f_correlation])

        return f_glcm
    

    def lbp_feature(self, img):
        radius = 3
        n_points = 8 * radius
        lbp = local_binary_pattern(img, n_points, radius, method='uniform')
        
        f_lbp, _ = np.histogram(lbp.ravel(),
                                bins=np.arange(0, n_points + 3),
                                range=(0, n_points + 2),
                                density=True)
        return f_lbp
    

    def hog_feature(self, img):
        img_norm = img / 255.0 if img.max() > 1.0 else img

        f_hog = hog(img_norm, 
                    orientations=9, 
                    pixels_per_cell=(16, 16),
                    cells_per_block=(2, 2), 
                    transform_sqrt=True, 
                    block_norm='L2-Hys',
                    visualize=False)

        return f_hog
    
    def extract_all_features(self, img):
        # f_stats = self.statistical_feature(img)
        f_glcm = self.glcm_feature(img)
        f_lbp = self.lbp_feature(img)
        f_hog = self.hog_feature(img)
    
        combined_features = np.hstack([f_glcm, f_lbp, f_hog])
        
        return combined_features

In [ ]:
x_features = []
extractor = ExtractFeature()

for img in tqdm(x_preprocessed):
    features = extractor.extract_all_features(img)

    x_features.append(features)

In [ ]:
def visualize_features(img, title="Feature Visualization"):
    if len(img.shape) > 2:
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    else:
        gray = img

    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')

    _, hog_image = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), visualize=True, transform_sqrt=True)

    plt.figure(figsize=(15, 5))

    # Gambar Asli
    plt.subplot(1, 3, 1)
    plt.imshow(gray, cmap='gray')
    plt.title("Original Preprocessed")
    plt.axis('off')

    # LBP
    plt.subplot(1, 3, 2)
    plt.imshow(lbp, cmap='gray')
    plt.title(f"LBP (Radius={radius})")
    plt.axis('off')

    # HOG
    plt.subplot(1, 3, 3)
    plt.imshow(hog_image, cmap='gray')
    plt.title("HOG Visualization")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

visualize_features(x_preprocessed[2]) 

<h1>Preparing Training Data</h1>

In [ ]:
# preparing labels and data
x_features = np.array(x_features)
y = np.array(y)


# splitting data training and test

x_train, x_test, y_train, y_test = train_test_split(x_features, y, random_state=42)

scalerx = StandardScaler()

x_train_scaled = scalerx.fit_transform(x_train)
x_test_scaled = scalerx.transform(x_test)



<h1>Training Model SVC</h1>

In [ ]:
svc_model = SVC(C=1, kernel='rbf')
tqdm(svc_model.fit(X=x_train_scaled, y=y_train))

<h1>Evaluate Model SVC</h1>

In [ ]:
y_pred = svc_model.predict(x_test_scaled)

acc = accuracy_score(y_test, y_pred)

In [ ]:
print(f"Accuracy model: {acc}")

print(classification_report(y_pred=y_pred, y_true=y_test))
cf_matrix = confusion_matrix(y_test, y_pred)
class_labels = loader.mapLabel()
disp = ConfusionMatrixDisplay(cf_matrix, display_labels=list(class_labels.values()))

print("Confusion Matrix:")
disp.plot(cmap='cool')
plt.show()

In [ ]:
# filename = "svc_bt_classification_model(128px).joblib"
# joblib.dump(svc_model, filename=filename)
# print(f"Saving model with name: {filename}")

In [ ]:
# # 2. Save the Scaler (CRITICAL STEP)
# joblib.dump(scalerx, 'scaler.joblib')

# # 3. Save the Class Mapping (so the app knows 0='glioma', etc.)
# # Assuming loader.int_to_class exists from your previous code
# joblib.dump(loader.int_to_class, 'class_labels.joblib')

<h1>Tumor Detection Model</h1>

In [ ]:
class TumorDetector:
    def __init__(self):
        pass

    def detect_via_threshold(self, img):
        """
        Detects tumor using Otsu's Thresholding and Contour detection.
        """
        # 1. Convert to Grayscale if strictly needed (though usually passed as preprocessed)
        if len(img.shape) == 3:
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        else:
            gray = img.copy()

        # 2. Gaussian Blur to reduce high frequency noise
        blur = cv.GaussianBlur(gray, (5, 5), 0)

        # 3. Thresholding (Otsu's method automatically finds the best threshold)
        thresh_val, binary = cv.threshold(blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

        # 4. Morphological Operations to remove small noise spots
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
        binary = cv.morphologyEx(binary, cv.MORPH_OPEN, kernel, iterations=2)
        
        # 5. Find Contours
        contours, _ = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        
        best_box = None
        max_area = 0

        for cnt in contours:
            area = cv.contourArea(cnt)
            # Filter: ignore very small noise and (optional) very large skull rings if needed
            if area > 500: 
                if area > max_area:
                    max_area = area
                    x, y, w, h = cv.boundingRect(cnt)
                    best_box = (x, y, w, h)
        
        return best_box, binary

    def detect_via_kmeans(self, img, k=3):
        """
        Detects tumor using K-Means Clustering to segment tissue types.
        Uses the 'KMeans' library you imported.
        """
        # 1. Reshape image to a list of pixels
        # if RGB: (H*W, 3), if Gray: (H*W, 1)
        if len(img.shape) == 3:
            pixel_values = img.reshape((-1, 3))
        else:
            pixel_values = img.reshape((-1, 1))
            
        pixel_values = np.float32(pixel_values)

        # Apply K-Means
        # k=3 usually separates: Background (Black), Brain (Grey), Tumor/Skull (White)
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.2)
        _, labels, centers = cv.kmeans(pixel_values, k, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

        # Convert back to image shape
        centers = np.uint8(centers)
        segmented_data = centers[labels.flatten()]
        segmented_image = segmented_data.reshape(img.shape[:2])

        # Find which label corresponds to the highest intensity center
        max_intensity_idx = np.argmax(centers) 
        
        # Create a binary mask for just that cluster
        mask = (labels.flatten() == max_intensity_idx).reshape(img.shape[:2]).astype(np.uint8) * 255

        # Clean mask and find contours 
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
        mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel, iterations=1)
        
        contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        
        best_box = None
        max_area = 0
        
        for cnt in contours:
            area = cv.contourArea(cnt)
            if area > 300: # Minimum area threshold
                if area > max_area:
                    max_area = area
                    x, y, w, h = cv.boundingRect(cnt)
                    best_box = (x, y, w, h)
                    
        return best_box, mask

In [ ]:
detector = TumorDetector()

# Pick a sample image from your dataset (e.g., from x_preprocessed or raw x)
sample_img = x[10] # Using original image for better visualization colors
# sample_img = x_preprocessed[10] # Or use preprocessed

# Detect
box, mask = detector.detect_via_threshold(sample_img)
# box, mask = detector.detect_via_kmeans(sample_img) # Try this if threshold fails

if box is not None:
    (x_box, y_box, w_box, h_box) = box
    
    # Draw the rectangle on a copy of the image
    result_img = sample_img.copy()
    cv.rectangle(result_img, (x_box, y_box), (x_box + w_box, y_box + h_box), (0, 255, 0), 2)
    
    # Visualization
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 3, 1)
    plt.imshow(sample_img, cmap='gray')
    plt.title("Original")
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask, cmap='gray')
    plt.title("Segmentation Mask")
    
    plt.subplot(1, 3, 3)
    plt.imshow(result_img, cmap='gray')
    plt.title("Detected Bounding Box")
    
    plt.show()
else:
    print("No tumor detected (contour area might be too small).")